# 3. Linear regression

## 3.6. Lab: Linear Regression

In [2]:
# -> Define spark session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear regression').getOrCreate()

In [3]:
import utils 

import pyspark.sql.functions as F

from pyspark.ml.regression import GeneralizedLinearRegression, LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

### *3.6.2 Simple Linear Regression*

In [4]:
# -> Load Boston dataset:

Boston = spark.read.csv('data/Boston.csv',header=True,inferSchema=True)

print('\nBoston dataset:')
Boston.show(3)

print('\nData types:')
Boston.printSchema()

# -> Prepare data:

data = utils.prepare_data(df = Boston,
                    labelCol = 'medv',
                    label_is_categorical = False,
                    categoricalCols = [],
                    continuousCols = ['lstat']
                   )
print('Prepared data for analysis:')
data.show(5)

# -> Describe the model using 'LinearRegression':

model = LinearRegression(featuresCol="features", labelCol='label')

# -> Fit the model:

model_fit = model.fit(data)

# -> Get model's summary:

utils.get_linear_modelSummary(model_fit)

# ----------------------------------------------------------------------------

# An alternative way to fit linear regression is decribe 
# it as Generalized Linear Regression using 'Generalized Linear Regression':

# -> Describe the model:
model = GeneralizedLinearRegression(family="gaussian", link="identity",featuresCol='features', labelCol='label')

# -> Fit the model:
model_fit = model.fit(data)

# -> Get model's summary:
print('\n')
utils.get_GLM_modelSummary(model_fit)

# ----------------------------------------------------------------------------

# -> Predictions:

print('\nPredictions:')
model_fit.transform(data).show(5)

# -> Predictions using new data:
new_data = spark.createDataFrame([
    (Vectors.dense([5.0,]),),
    (Vectors.dense([10.0,]),),
    (Vectors.dense([15.0,]),)
], ["features"])

print('\nPredictions using new data:')
model_fit.transform(new_data).show(5)


Boston dataset:
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio| black|lstat|medv|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575|65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421|78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 3 rows


Data types:
root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = tru

### 3.6.3 *Multiple Linear Regression*

In [6]:
# -> Prepare the data:

data = utils.prepare_data(df = Boston,
                    labelCol = 'medv',
                    label_is_categorical = False,
                    categoricalCols = [],
                    continuousCols = [col for col in Boston.columns if col != 'medv']
                   )

# -> Describe the model:

model = LinearRegression(featuresCol="features", labelCol='label')

# -> Fit the model:

model_fit = model.fit(data)

# -> Get model's summary:

utils.get_linear_modelSummary(model_fit)

# -> Estimate VIF values:

print(utils.estimateVIF(df=Boston, Cols=[col for col in Boston.columns if col != 'medv']))

# -> Estimate correlation matrix to detect highly correlated features:

utils.estimate_correlation_matrix(df=data, 
                            cols=[col for col in Boston.columns if col != 'medv'], 
                            method='pearson', 
                            round_decimals=3)

Estimated model results:
## -------------------------------------------------
##  Estimate | Std.Error | t Values | P-value
##     -0.108      0.033   -3.287      0.001
##      0.046      0.014    3.382      0.001
##      0.021      0.061    0.334      0.738
##      2.687      0.862    3.118      0.002
##    -17.767      3.820   -4.651      0.000
##      3.810      0.418    9.116      0.000
##      0.001      0.013    0.052      0.958
##     -1.476      0.199   -7.398      0.000
##      0.306      0.066    4.613      0.000
##     -0.012      0.004   -3.280      0.001
##     -0.953      0.131   -7.283      0.000
##      0.009      0.003    3.467      0.001
##     -0.525      0.051  -10.347      0.000
##     36.459      5.103    7.144      0.000
## ---
## Mean squared error: 21.895, RMSE: 4.679
## Multiple R-squared: 0.741, Total iterations:1.
      Feature       VIF
Rank                   
0         tax  9.008554
1         rad  7.484496
2         nox  4.393720
3       indus  3.991596
4 

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
crim,1,-0.2,0.407,-0.056,0.421,-0.219,0.353,-0.38,0.626,0.583,0.29,-0.385,0.456
zn,-0.2,1,-0.534,-0.043,-0.517,0.312,-0.57,0.664,-0.312,-0.315,-0.392,0.176,-0.413
indus,0.407,-0.534,1,0.063,0.764,-0.392,0.645,-0.708,0.595,0.721,0.383,-0.357,0.604
chas,-0.056,-0.043,0.063,1,0.091,0.091,0.087,-0.099,-0.007,-0.036,-0.122,0.049,-0.054
nox,0.421,-0.517,0.764,0.091,1,-0.302,0.731,-0.769,0.611,0.668,0.189,-0.38,0.591
rm,-0.219,0.312,-0.392,0.091,-0.302,1,-0.24,0.205,-0.21,-0.292,-0.356,0.128,-0.614
age,0.353,-0.57,0.645,0.087,0.731,-0.24,1,-0.748,0.456,0.506,0.262,-0.274,0.602
dis,-0.38,0.664,-0.708,-0.099,-0.769,0.205,-0.748,1,-0.495,-0.534,-0.232,0.292,-0.497
rad,0.626,-0.312,0.595,-0.007,0.611,-0.21,0.456,-0.495,1,0.91,0.465,-0.444,0.489
tax,0.583,-0.315,0.721,-0.036,0.668,-0.292,0.506,-0.534,0.91,1,0.461,-0.442,0.544


## *3.6.4 Interaction Terms*

In [7]:
# -> Estimate interaction values:

data = Boston.withColumn('lstat_age_interaction', F.col('lstat') * F.col('age'))

# -> Prepare the data:

data = utils.prepare_data(df = data,
                    labelCol = 'medv',
                    label_is_categorical = False,
                    categoricalCols = [],
                    continuousCols = ['lstat', 'lstat_age_interaction']
                   )

# -> Describe the model:

model = LinearRegression(featuresCol="features", labelCol='label')

# -> Fit the model:

model_fit = model.fit(data)

# -> Get model's summary:

utils.get_linear_modelSummary(model_fit)

Estimated model results:
## -------------------------------------------------
##  Estimate | Std.Error | t Values | P-value
##     -1.388      0.127  -10.938      0.000
##      0.004      0.001    3.621      0.000
##     36.042      0.691   52.133      0.000
## ---
## Mean squared error: 37.505, RMSE: 6.124
## Multiple R-squared: 0.556, Total iterations:1.


## *3.6.5 Non-linear Transformations of the Predictors*

In [9]:
# -> Estimate polynomial values
data = Boston.withColumn('lstat_power_2', F.pow(F.col('lstat'),2))

# -> Prepare the data:
data = utils.prepare_data(df = data,
                    labelCol = 'medv',
                    label_is_categorical = False,
                    categoricalCols = [],
                    continuousCols = ['lstat', 'lstat_power_2']
                   )

# -> Describe the model:
model = LinearRegression(featuresCol="features", labelCol='label')

# -> Fit the model:
model_fit = model.fit(data)

# -> Get model's summary:
utils.get_linear_modelSummary(model_fit)

## -> Compare nested models:

# -> Estimate polynomial values
data = Boston.withColumn('label', F.col('medv')).withColumn('lstat_power_2', F.pow(F.col('lstat'),2))

# -> Assembler:
assembler_1 = VectorAssembler(inputCols=["lstat"], outputCol="features_without_power")
assembler_2 = VectorAssembler(inputCols=["lstat", 'lstat_power_2'], outputCol="features_with_power")

# -> Pipeline:
pipeline = Pipeline(stages=[assembler_1, assembler_2])
data = pipeline.fit(data).transform(data)

# -> Compare nested models

model_1 = LinearRegression(featuresCol='features_without_power', labelCol='label')
model_2 = LinearRegression(featuresCol='features_with_power', labelCol='label')

model_1_fit = model_1.fit(data)
model_2_fit = model_2.fit(data)

print('MSE of the model with lstat: {:.3f}'.format(model_1_fit.summary.meanSquaredError))
print('MSE of the model with lstat and lstat^2: {:.3f}'.format(model_2_fit.summary.meanSquaredError))

Estimated model results:
## -------------------------------------------------
##  Estimate | Std.Error | t Values | P-value
##     -2.333      0.124  -18.843      0.000
##      0.044      0.004   11.628      0.000
##     42.862      0.872   49.149      0.000
## ---
## Mean squared error: 30.331, RMSE: 5.507
## Multiple R-squared: 0.641, Total iterations:1.
MSE of the model with lstat: 38.483
MSE of the model with lstat and lstat^2: 30.331


## *3.6.6 Qualitative Predictors*

In [11]:
# -> Load and clear Carseats dataset:

Carseats = spark.read.csv('data/Carseats.csv',header=True,inferSchema=True)
Carseats = Carseats.na.drop(how='any')
Carseats = Carseats.withColumn('High', F.when(F.col('Sales')<=8,'No').otherwise('Yes'))

print('\nCarseat dataset:'); Carseats.show(5)
print('\nData types:'); Carseats.printSchema()

# -> Prepare dataset:

data = utils.prepare_data(df=Carseats,
                   labelCol='Sales',
                   label_is_categorical = False,
                   categoricalCols=['ShelveLoc', 'Urban', 'US', 'High'],
                   continuousCols=['CompPrice','Income', 'Advertising', 'Population', 'Price', 'Age', 'Education'])

print('\nPrepared data for ML analysis:')
data.select('label', 'features').show(5, truncate=False)

# -> Describe the model

model = LinearRegression(featuresCol="features", labelCol='Sales')

# -> Fit the model

model_fit = model.fit(data)

# -> Print model's results:

utils.get_linear_modelSummary(model_fit)


Carseat dataset:
+---+-----+---------+------+-----------+----------+-----+---------+---+---------+-----+---+----+
|_c0|Sales|CompPrice|Income|Advertising|Population|Price|ShelveLoc|Age|Education|Urban| US|High|
+---+-----+---------+------+-----------+----------+-----+---------+---+---------+-----+---+----+
|  1|  9.5|      138|    73|         11|       276|  120|      Bad| 42|       17|  Yes|Yes| Yes|
|  2|11.22|      111|    48|         16|       260|   83|     Good| 65|       10|  Yes|Yes| Yes|
|  3|10.06|      113|    35|         10|       269|   80|   Medium| 59|       12|  Yes|Yes| Yes|
|  4|  7.4|      117|   100|          4|       466|   97|   Medium| 55|       14|  Yes|Yes|  No|
|  5| 4.15|      141|    64|          3|       340|  128|      Bad| 38|       13|  Yes| No|  No|
+---+-----+---------+------+-----------+----------+-----+---------+---+---------+-----+---+----+
only showing top 5 rows


Data types:
root
 |-- _c0: integer (nullable = true)
 |-- Sales: double (nullable =